# Figure 4b

Energy as a function of time.

In [20]:
import os
import sys

import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import clear_output

import matplotlib as mpl 
import matplotlib.pyplot as plt

sys.path.insert(0, '../../icenumerics/')
sys.path.insert(0, './auxnumerics/')
import icenumerics as ice

from numba import jit
import auxiliary as aux
import montecarlo_tools as mc
import chirality_tools as chir
import energy as eg
ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

/mnt/c/Users/leoga/Documents/GitHub/stuckgs/src/./auxnumerics/energy.py:23: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_min_from_domain(f,domain):
/mnt/c/Users/leoga/Documents/GitHub/stuckgs/src/./auxnumerics/energy.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_pbc_distance(L,xi,xj):
/mnt/c/Users/leoga/Documents/GitHub/stuckgs/src/./auxn

In [16]:
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# Parameters

In [17]:
quench_time = 300*ureg.s
evolution_time = 60*ureg.s
total_time = quench_time + evolution_time
#sim_path = '/mnt/e/stuckgs/data/sims'
sim_path = '../data/sims/'

params = {
    "particle_radius":1.4*ureg.um,
    "particle_susceptibility":0.4,
    "particle_diffusion":0.14*ureg.um**2/ureg.s,
    "particle_temperature":300*ureg.K,
    "particle_density":1000*ureg.kg/ureg.m**3,

    "trap_sep":3*ureg.um,
    "trap_height":8*ureg.pN*ureg.nm,
    "trap_stiffness":100e-3*ureg.pN/ureg.nm,
    "height_spread":0,
    "susceptibility_spread":0,
    "isperiodic":True,

    "total_time":total_time,
    "framespersec":20*ureg.Hz,
    "dt":0.1*ureg.ms,
    "max_field":10*ureg.mT,
    "sim_temp":300*ureg.K,
    "sim_dipole_cutoff":40*ureg.um,
}

params["lattice_constant"] = params["trap_sep"]+(2*params["particle_radius"]+1*ureg.um)*np.sqrt(2)
params['size'] = 30

params['mu0'] = (4*np.pi)*1e-7 * ureg.H/ureg.m
params['m'] = np.pi * (2*params['particle_radius'])**3 *params['particle_susceptibility']*params['max_field']/6/params['mu0']
params['kb'] = 1.380649e-23 * ureg.J / ureg.K
params['kbT'] = (params['kb'] * params['sim_temp']).to(ureg.nm * ureg.pN)
params['freedom'] = - (params['mu0']*params['m']**2/4/np.pi).to(ureg.pN * ureg.nm * ureg.um**3).magnitude

In [4]:
realization = 1
frame = 0
trj = pd.read_csv(os.path.join(sim_path,'30','trj',f'trj{realization}.csv'), index_col=['frame','id'])
trj

type          x        y    z           mux  muy  muz       t
frame id                                                                 
0     1        1    2.68701    0.000  0.0  0.000000e+00  0.0  0.0    0.00
      2        1   11.06100    0.000  0.0  0.000000e+00  0.0  0.0    0.00
      3        1   22.43500    0.000  0.0  0.000000e+00  0.0  0.0    0.00
      4        1   30.80900    0.000  0.0  0.000000e+00  0.0  0.0    0.00
      5        1   39.18310    0.000  0.0  0.000000e+00  0.0  0.0    0.00
...          ...        ...      ...  ...           ...  ...  ...     ...
8399  3596     2  209.35000  247.033  0.0 -1.836970e-16 -3.0 -0.0  419.95
      3597     2  217.72400  247.033  0.0  1.836970e-16  3.0  0.0  419.95
      3598     2  226.09800  247.033  0.0  1.836970e-16  3.0  0.0  419.95
      3599     2  234.47200  247.033  0.0 -1.836970e-16 -3.0 -0.0  419.95
      3600     2  242.84600  247.033  0.0 -1.836970e-16 -3.0 -0.0  419.95

[30240000 rows x 8 columns]

In [42]:
frames = trj.index.get_level_values('frame').unique().to_list()
sframes = frames[:1]
energy = [aux.calculate_energy(params,aux.get_coordinates_at_frame(trj,frame)) for frame in sframes]
energy

[-3234141.59659486]

In [18]:
L = params['size']*params['lattice_constant'].magnitude
magic_number = params['freedom']

In [ ]:
eg.calculate_energy(magic_number,L,eg.get_coordinates_at_frame(trj,0))

In [50]:
sel_particles = aux.get_coordinates_at_frame(trj,0).to_numpy()
eg.precompiled_calculate_energy(magic_number,L,sel_particles)

-3234141.59659486